# Assignment
### Dependencies
Using Python 3.7

    pip install ipython
    pip install pandas
    pip install matplotlib
    pip install -U scikit-learn
    pip install jupyter notebook
    
### Import functions

In [21]:
from tools.helpers import read_json, pandas_keep_columns, retrieve_data, create_project_workspace
from assignment import setup_workspace_retrieve_and_parse_data, select_historic_data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 5]

### Read Project Configuration file
This contains download link, file names and columns to use. 

In [13]:
data = read_json('data/project_config.json')
steps = data['steps']

# Part 1 Data Exploration and Evaluation
- [x] Clean columns, carry forward
`['loan_amnt’, 'funded_amnt’, 'term’, 'int_rate’, 'grade’, 'annual_inc’, 'issue_d’, 'dti’, 'revol_bal’, 'total_pymnt’, 'loan_status’]`

- [x] Perform any necessary cleaning and aggregations to explore and better
  understand the dataset.
  - [x] Describe the `data.describe()`

- [x] Describe any assumptions you made to handle null variables and outliers.
    - [x] Remove outliers for Annual Income
    - [x] Remove outliers for DTI
    - [x] Total credit revolving balance (revol_bal)

- [x] Describe the distributions of the features.
    - [x] Include two data visualizations and 
    - [x] Two summary statistics to support these findings.

In [9]:
# df = setup_workspace_retrieve_and_parse_data(data)

Three columns have outliers in the distribution:
* Annual Income
* DTI
* Total credit revolving balance

## Variables with Outliers
#### Annual Income
![annual_inc](img/pre_filter/annual_inc.png)
#### DTI
![dti](img/pre_filter/dti.png)
#### Total credit revolving balance
![revol_bal](img/pre_filter/revol_bal.png)

Rows with >= 3 or <= -3 Z-score were removed. The following are the filtered histograms;

## Variables filtered Outliers
#### Annual Income
![annual_inc](img/annual_inc.png)
#### DTI
![dti](img/dti.png)
#### Total credit revolving balance
![revol_bal](img/revol_bal.png)

In [16]:
df = pd.read_csv(
    'data/processing/data_filtered.csv',
    low_memory=False,
    parse_dates=['issue_date']
)

df.describe()

,loan_amnt,funded_amnt,int_rate,annual_inc,dti,revol_bal,total_pymnt,issue_year,issue_month,months,default,annualized_rate_return,z_score_annual_inc,z_score_revol_bal,z_score_dti
count,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06,2.213821e+06
mean,1.487242e+04,1.486724e+04,1.310676e+01,7.508537e+04,1.841688e+01,1.473499e+04,1.172318e+04,2.015848e+03,6.750350e+00,3.065078e+01,5.375566e-01,-1.076321e-01,-2.579554e-02,-8.097000e-02,-2.707846e-02
std,9.074879e+03,9.072984e+03,4.825068e+00,4.369579e+04,8.864596e+00,1.272795e+04,9.790934e+03,1.793568e+00,3.408234e+00,2.162006e+01,4.985876e-01,1.996257e-01,3.877308e-01,5.949982e-01,6.292046e-01
min,5.000000e+02,5.000000e+02,5.310000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,2.007000e+03,1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-6.920591e-01,-7.697921e-01,-1.405279e+00
25%,8.000000e+03,8.000000e+03,9.490000e+00,4.600000e+04,1.186000e+01,5.874000e+03,4.252910e+03,2.015000e+03,4.000000e+00,1.200000e+01,0.000000e+00,-2.506934e-01,-2.838821e-01,-4.951981e-01,-4.924827e-01
50%,1.250000e+04,1.250000e+04,1.262000e+01,6.500000e+04,1.777000e+01,1.112500e+04,9.008950e+03,2.016000e+03,7.000000e+00,2.900000e+01,1.000000e+00,3.957914e-03,-1.152872e-01,-2.497277e-01,-7.299380e-02
75%,2.000000e+04,2.000000e+04,1.599000e+01,9.100000e+04,2.437000e+01,1.962600e+04,1.656851e+04,2.017000e+03,1.000000e+01,4.400000e+01,1.000000e+00,4.367308e-02,1.154216e-01,1.476717e-01,3.954709e-01
max,4.000000e+04,4.000000e+04,3.099000e+01,4.160000e+05,6.106000e+01,8.063900e+04,6.329688e+04,2.018000e+03,1.200000e+01,1.380000e+02,1.000000e+00,2.947122e-01,2.999281e+00,2.999869e+00,2.999709e+00


# Part 2 Business Analysis
- [x] Assume a 36 month investment period for each loan, and exclude loans with less than 36 months of data available.

- [x] What percentage of loans has been fully paid?
    * **0.7562866221941765**

- [x] When bucketed by year of origination and grade, which cohort has the highest rate of defaults? Here you may assume that any loan which was not fully paid had “defaulted”.

- [x] When bucketed by year of origination and grade, what annualized rate of return have these loans generated on average?
      
    * For simplicity, use the following approximation:
      `Annualized rate of return = (total_pymnt / funded_amnt) ^ (1/3) - 1`


In [22]:
select_historic_data(df)

TypeError: select_historic_data() takes 0 positional arguments but 1 was given